In [5]:
pip install imbalanced-learn

     |████████████████████████████████| 199 kB 20.5 MB/s eta 0:00:01
     |████████████████████████████████| 30.8 MB 54.2 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 78.5 MB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 0.17.0
    Uninstalling joblib-0.17.0:
      Successfully uninstalled joblib-0.17.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lale 0.6.10 requires scikit-learn<=1.0.2, but you have scikit-learn 1.1.3 which is incompatible.
autoai-ts-libs 1.1.9 requires scikit-learn<1.1,>=1.0.2; python_version >= "3.9", but you have scikit-learn 1.1.3 which is incompatible.
autoai-libs 1.13.6 requires scikit-learn<1.1,>=0.20.3

In [7]:
import numpy as np 
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import  GradientBoostingClassifier


import warnings
warnings.filterwarnings('ignore')

In [14]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='4VwY4G6okycGym38fd4U1yrQ3NMMCI6LjXzPfFCyJEfw',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'smartlenderapplicantcredibilitypr-donotdelete-pr-anlrfdagcsms2f'
object_key = 'loan_prediction.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df= pd.read_csv(body)
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [15]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [16]:
numerical_features = df.select_dtypes(include = [np.number]).columns
categorical_features = df.select_dtypes(include = [np.object]).columns

In [17]:
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])
df['Married'] = df['Married'].fillna(df['Married'].mode()[0])
df['Dependents'] = df['Dependents'].str.replace('+','')
df['Dependents'] = df['Dependents'].fillna(df['Dependents'].mode()[0])
df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].mode()[0])
df['Self_Employed'] = df['Self_Employed'].fillna(df['Self_Employed'].mode()[0])
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0])
df['Credit_History'] = df['Credit_History'].fillna(df['Credit_History'].mode()[0])

In [18]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [19]:
df['Gender'].replace({'Male':1,'Female':0},inplace=True)
df['Dependents'].replace({'0':0,'1':1,'2':2,'3':3},inplace=True)
df['Married'].replace({'Yes':1,'No':0},inplace=True)
df['Self_Employed'].replace({'Yes':1,'No':0},inplace=True)
df['Property_Area'].replace({'Urban':2,'Rural':0,'Semiurban':1},inplace=True)
df['Education'].replace({'Graduate':1,'Not Graduate':0},inplace=True)
df['Loan_Status'].replace({'Y':1,'N':0},inplace=True)

In [20]:
df['CoapplicantIncome']=df['CoapplicantIncome'].astype("int64")
df['LoanAmount']=df['LoanAmount'].astype("int64")
df['Loan_Amount_Term']=df['Loan_Amount_Term'].astype("int64")
df['Credit_History']=df['Credit_History'].astype("int64")

In [21]:
le = LabelEncoder()
df['Loan_ID'] = le.fit_transform(df.Loan_ID)
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,0,0,1,0,5849,0,120,360,1,2,1
1,1,1,1,1,1,0,4583,1508,128,360,1,0,0
2,2,1,1,0,1,1,3000,0,66,360,1,2,1
3,3,1,1,0,0,0,2583,2358,120,360,1,2,1
4,4,1,0,0,1,0,6000,0,141,360,1,2,1


In [22]:
smote = SMOTETomek(0.90)

y = df['Loan_Status']
x = df.drop(columns=['Loan_Status'],axis=1)
x_bal,y_bal = smote.fit_resample(x,y)
print(y.value_counts())
print(y_bal.value_counts())

1    422
0    192
Name: Loan_Status, dtype: int64
1    372
0    329
Name: Loan_Status, dtype: int64


In [23]:
scaler = StandardScaler()
x_bal = scaler.fit_transform(x_bal)
x_bal = pd.DataFrame(x_bal)
x_bal.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-1.779917,0.521759,-1.186986,-0.693749,0.622992,-0.330420,0.107509,-0.561748,-0.307269,0.270188,0.634035,1.405574
1,-1.774126,0.521759,0.842470,0.365768,0.622992,-0.330420,-0.121510,-0.013003,-0.199057,0.270188,0.634035,-1.201059
2,-1.768335,0.521759,0.842470,-0.693749,0.622992,3.026453,-0.407875,-0.561748,-1.037697,0.270188,0.634035,1.405574
3,-1.762544,0.521759,0.842470,-0.693749,-1.605158,-0.330420,-0.483311,0.296302,-0.307269,0.270188,0.634035,1.405574
4,-1.756753,0.521759,-1.186986,-0.693749,0.622992,-0.330420,0.134825,-0.561748,-0.023213,0.270188,0.634035,1.405574


In [24]:
x_train, x_test, y_train, y_test = train_test_split(x_bal, y_bal, test_size = 0.33, random_state = 42)

# Compare the Models - KNN, Decision Tree, Random Forest CLassifier, XGBoost

## KNN

In [25]:
def kneighborsClassifier(x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier() 
    knn.fit(x_train,y_train)
    yPred = knn.predict(x_test)
    print("****KNN****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=knn.predict(x_test)
    y_pred1=knn.predict(x_train)
    knn_test_accuracy = accuracy_score(y_test,y_pred)
    knn_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ',knn_test_accuracy)
    print('Training accuracy: ', knn_train_accuracy)

kneighborsClassifier(x_train, x_test, y_train, y_test)

****KNN****
Confusion matrix
[[ 70  39]
 [ 12 111]]
Classification report
              precision    recall  f1-score   support

           0       0.85      0.64      0.73       109
           1       0.74      0.90      0.81       123

    accuracy                           0.78       232
   macro avg       0.80      0.77      0.77       232
weighted avg       0.79      0.78      0.78       232

Testing accuracy:  0.7801724137931034
Training accuracy:  0.8400852878464818


# Decision Tree

In [26]:
def decisionTreeClassifier(x_train, x_test, y_train, y_test):
    dt = DecisionTreeClassifier() 
    dt.fit(x_train,y_train)
    yPred = dt.predict(x_test)
    print("****DecisionTreeClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=dt.predict(x_test)
    y_pred1=dt.predict(x_train)
    decision_tree_test_accuracy = accuracy_score(y_test,y_pred)
    decision_tree_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', decision_tree_test_accuracy)
    print('Training accuracy: ', decision_tree_train_accuracy)

decisionTreeClassifier(x_train, x_test, y_train, y_test)

****DecisionTreeClassifier****
Confusion matrix
[[80 29]
 [31 92]]
Classification report
              precision    recall  f1-score   support

           0       0.72      0.73      0.73       109
           1       0.76      0.75      0.75       123

    accuracy                           0.74       232
   macro avg       0.74      0.74      0.74       232
weighted avg       0.74      0.74      0.74       232

Testing accuracy:  0.7413793103448276
Training accuracy:  1.0


## Random Forest CLassifier

In [27]:
def randomForestClassifier(x_train, x_test, y_train, y_test):
    rf = RandomForestClassifier() 
    rf.fit(x_train,y_train)
    yPred = rf.predict(x_test)
    print("****RandomForestClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=rf.predict(x_test)
    y_pred1=rf.predict(x_train)
    random_forest_test_accuracy = accuracy_score(y_test,y_pred)
    random_forest_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', random_forest_test_accuracy)
    print('Training accuracy: ',random_forest_train_accuracy)

randomForestClassifier(x_train, x_test, y_train, y_test)

****RandomForestClassifier****
Confusion matrix
[[ 79  30]
 [ 11 112]]
Classification report
              precision    recall  f1-score   support

           0       0.88      0.72      0.79       109
           1       0.79      0.91      0.85       123

    accuracy                           0.82       232
   macro avg       0.83      0.82      0.82       232
weighted avg       0.83      0.82      0.82       232

Testing accuracy:  0.8232758620689655
Training accuracy:  1.0


# XGBoost

In [28]:
def xgboost(x_train, x_test, y_train, y_test):
    xg = GradientBoostingClassifier() 
    xg.fit(x_train,y_train)
    yPred = xg.predict(x_test)
    print("****Gradient BoostingClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=xg.predict(x_test)
    y_pred1=xg.predict(x_train)
    xgboost_test_accuracy = accuracy_score(y_test,y_pred)
    xgboost_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', xgboost_test_accuracy)
    print('Training accuracy: ', xgboost_train_accuracy)
  
xgboost(x_train, x_test, y_train, y_test)

****Gradient BoostingClassifier****
Confusion matrix
[[ 77  32]
 [ 13 110]]
Classification report
              precision    recall  f1-score   support

           0       0.86      0.71      0.77       109
           1       0.77      0.89      0.83       123

    accuracy                           0.81       232
   macro avg       0.82      0.80      0.80       232
weighted avg       0.81      0.81      0.80       232

Testing accuracy:  0.8060344827586207
Training accuracy:  0.9488272921108742


In [29]:
kneighborsClassifier(x_train, x_test, y_train, y_test)
decisionTreeClassifier(x_train, x_test, y_train, y_test)
randomForestClassifier(x_train, x_test, y_train, y_test)
xgboost(x_train, x_test, y_train, y_test)

****KNN****
Confusion matrix
[[ 70  39]
 [ 12 111]]
Classification report
              precision    recall  f1-score   support

           0       0.85      0.64      0.73       109
           1       0.74      0.90      0.81       123

    accuracy                           0.78       232
   macro avg       0.80      0.77      0.77       232
weighted avg       0.79      0.78      0.78       232

Testing accuracy:  0.7801724137931034
Training accuracy:  0.8400852878464818
****DecisionTreeClassifier****
Confusion matrix
[[80 29]
 [24 99]]
Classification report
              precision    recall  f1-score   support

           0       0.77      0.73      0.75       109
           1       0.77      0.80      0.79       123

    accuracy                           0.77       232
   macro avg       0.77      0.77      0.77       232
weighted avg       0.77      0.77      0.77       232

Testing accuracy:  0.771551724137931
Training accuracy:  1.0
****RandomForestClassifier****
Confusion matr

**KNN**
1.   Testing accuracy:  0.7955555555555556
2.   Training accuracy:  0.8392070484581498

**DecisionTreeClassifier**
1. Testing accuracy:  0.7822222222222223
2. Training accuracy:  1.0

**RandomForestClassifier**
1. Testing accuracy:  0.8133333333333334
2. Training accuracy:  1.0

**Gradient BoostingClassifier**
1. Testing accuracy:  0.8088888888888889
2. Training accuracy:  0.9405286343612335

RandomForestClassifier is chosen for further process.

# Evaluating Performance Of The Model And Saving The Model

In [30]:
from sklearn.model_selection import cross_val_score
forest_reg = RandomForestClassifier(n_estimators = 10, criterion = "entropy", random_state = 42) 
forest_reg.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [31]:
yPred = forest_reg.predict(x_test)
y_pred=forest_reg.predict(x_test)
y_pred1=forest_reg.predict(x_train)
random_forest_test_accuracy = accuracy_score(y_test,y_pred)
random_forest_train_accuracy = accuracy_score(y_train,y_pred1)
print('Testing accuracy: ', random_forest_test_accuracy)
print('Training accuracy: ',random_forest_train_accuracy)
f1_score(yPred,y_test, average='weighted')
cv = cross_val_score(forest_reg,x,y,cv=5)
np.mean(cv)

Testing accuracy:  0.8017241379310345
Training accuracy:  0.9914712153518124


0.723057443689191

In [21]:
!pip install ibm_watson_machine_learning
!pip install watson_machine_learning_client

     |████████████████████████████████| 538 kB 15.9 MB/s eta 0:00:01


In [49]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url" : "https://us-south.ml.cloud.ibm.com",
    "apikey" : "iSkSiRiEuV3YOJmWgpgqICsyt8pfDqGIXzBheNTjc3Y9"
}

client=APIClient(wml_credentials)

In [51]:
def guid_from_space_name(client, space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client,'models')
print("Space UID = "+space_uid)

In [ ]:
client.set.default_space(space_uid)

In [46]:
client.software_specifications.list()

You must set the space/project id. Use `client.set.default_space(<SPACE_UID>)`/`client.set.default_project(<PROJECT_UID>)` to proceed.


WMLClientError: You must set the space/project id. Use `client.set.default_space(<SPACE_UID>)`/`client.set.default_project(<PROJECT_UID>)` to proceed.

In [33]:
software_spec_uid = client.software_specifications.get_uid_by_name("default_py3.7")
software_spec_uid

'e4429883-c883-42b6-87a8-f419d64088cd'

In [35]:
model_details = client.repository.store_model(
    model=forest_reg,meta_props={
    client.repository.ModelMetaNames.NAME: "Loan_Prediction",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_0.20",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid }
                                           )
model_id = client.repository.get_model_uid(model_details)

Traceback (most recent call last):
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 254, in _save_scikit_pipeline_model
    model_artifact = self._create_pipeline_model(artifact, query_param)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 514, in _create_pipeline_model
    model_artifact = self._create_pipeline_model_v4_cloud(model_artifact, query_param)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 1005, in _create_pipeline_model_v4_cloud
    model_output = self.repository_api.ml_assets_model_creation_v4_cloud(model_input, query_param,headers)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/swagger_client/apis/repository_api.py", line 301,

WMLClientError: Publishing model failed.
Reason: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 13 Nov 2022 15:53:58 GMT', 'Content-Type': 'application/json', 'Content-Length': '292', 'Connection': 'keep-alive', 'server-timing': 'intid;desc=2951cdd70b527ad7, intid;desc=2951cdd70b527ad7', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'CF-Cache-Status': 'DYNAMIC', 'Server': 'cloudflare', 'CF-RAY': '7698aee8cb50e75e-DFW'})
HTTP response body: {
  "trace": "7c5cabc5fe8e83cfc13fa5f1ec31ad87",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported software specification 'default_py3.7'.",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}


In [ ]:
model_id